In [26]:
!python --version

Python 3.10.12


In [ ]:
!pip install urllib3==1.25.10
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.6/127.6 kB 1.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.4
    Uninstalling urllib3-1.25.4:
      Successfully uninstalled urllib3-1.25.4
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-3lv649j8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-3lv649j8
  Resolved https://github.com/huggingface/transformers to commit 35551f9a0f66a22de4971b4a51b3c172d3b87f95
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "sk", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "sk", split="test", use_auth_token=True)

print(common_voice)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 5246
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 2267
    })
})


In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 5246
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2267
    })
})


In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Slovak", task="transcribe")

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Slovak", task="transcribe")

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/56f266729c07f8b785e36e3483e268c1c12ff1d5628aef25f2807c2d0b84eded/sk_train_0/common_voice_sk_28451204.mp3', 'array': array([ 1.13686838e-13, -2.98427949e-13, -5.68434189e-13, ...,
        1.34686479e-06, -1.18348544e-06, -1.41438250e-06]), 'sampling_rate': 48000}, 'sentence': 'hlt; dúšok; prehltnutie'}


In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/56f266729c07f8b785e36e3483e268c1c12ff1d5628aef25f2807c2d0b84eded/sk_train_0/common_voice_sk_28451204.mp3', 'array': array([ 1.81898940e-12,  1.09139364e-11,  2.54658516e-11, ...,
       -3.16055389e-06, -2.85964279e-06,  1.54215627e-06]), 'sampling_rate': 16000}, 'sentence': 'hlt; dúšok; prehltnutie'}


In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/5246 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2267 [00:00<?, ? examples/s]

In [ ]:
common_voice.save_to_disk("common_voice")

Saving the dataset (0/11 shards):   0%|          | 0/5246 [00:00<?, ? examples/s]

Saving the dataset (0/5 shards):   0%|          | 0/2267 [00:00<?, ? examples/s]

In [ ]:
# Save your dataset to google drive
common_voice.save_to_disk(F"/content/drive/My Drive/SML-DLA_lab2/common_voice/")

Saving the dataset (0/11 shards):   0%|          | 0/5246 [00:00<?, ? examples/s]

Saving the dataset (0/5 shards):   0%|          | 0/2267 [00:00<?, ? examples/s]